# Scratchwork: Cayley-Dickson Algebras

The purpose of this notebook is to develop the functions required to perform the Cayley-Dickson construction within the ``finite_algebras`` module.

In the Python module, ``finite_algebras``, all elements are strings.

When direct products or Cayley-Dickson algebras are constructed, the component algebras' elements are concatentated (joined) using a string delimiter (':' by default).

So, operations like scalar multiplication, negation, conjugation, etc, on the newly constructed "compound" elements, require some string manipulation to accomplish.

## Test Algebras

In [1]:
import finite_algebras as alg

In [35]:
F3 = alg.generate_algebra_mod_n(3, elem_name='')
F3sqr = F3.sqr()
F3quad = F3sqr.sqr()

F4 = alg.generate_algebra_mod_n(4, elem_name='')
F4sqr = F4.sqr()

F5 = alg.generate_algebra_mod_n(5, elem_name='')
F5sqr = F5.sqr()

F7 = alg.generate_algebra_mod_n(7, elem_name='')
F7sqr = F7.sqr()

## Prototype Code

In [3]:
# =====================================================================
# Utilities for Squared Rings & Fields (i.e., Cayley-Dickson Algebras)
# =====================================================================

def cda_scalar_mult(scalar_name, elem_name, algebra):
    """ Scalar multiplication. 'a' * 'c:d' = 'a*c:a*d'
    Example: scalar_mult('2', '1:2', F3) ==> '2:1'
    """
    delimiter = algebra.direct_product_delimiter()
    components = elem_name.split(delimiter)
    return delimiter.join(map(lambda x: algebra.mult(scalar_name, x), components))


def cda_negate(elem_name, algebra):
    """ Negation:  -'a:b' = '-a:-b'
    Example: negate('1:2', F3) ==> '2:1'
    """
    delimiter = algebra.direct_product_delimiter()
    components = elem_name.split(delimiter)
    return delimiter.join(map(lambda x: algebra.inv(x), components))


def cda_conjugate(elem_name, algebra):
    """ Conjugation: conj('a:b') = 'a:-b'
    Example: conjugate('0:1', F3) ==> '0:2'
    """
    delimiter = algebra.direct_product_delimiter()
    components = elem_name.split(delimiter)
    head = components[0]
    tail = components[1:]
    tail_negated = list(map(lambda x: algebra.inv(x), tail))
    new_components = list(head) + tail_negated
    return delimiter.join(new_components)


def cda_sqr_abs_val(elem_name, algebra, alg_sqr):
    """Squared Absolute Value: 'a:b'^2 = 'a:b' * conj('a:b')
    Example: sqr_abs_val('1:2', F3, F3sqr) ==> '2'
    """
    delimiter = algebra.direct_product_delimiter()
    if delimiter in elem_name:
        # elem_name is not a scalar
        val = alg_sqr.mult(elem_name, cda_conjugate(elem_name, algebra))
    else:
        # elem_name is a scalar
        val = algebra.mult(elem_name, elem_name)
    comp = val.split(delimiter)
    return comp[0]


# NOTE: The inverse function below is just for comparison/verification.
# The Field method, mult_inv, is the better way to compute the inverse of an element.

def cda_inverse(elem_name, algebra, alg_sqr):
    """ Inversion: inv('a:b') = conj('a:b') / sqr_abs_val('a:b')
    Only works for Fields, not Rings.
    Example: inverse('1:1', F3, F3sqr) ==> '2:1'
    """
    if elem_name == alg_sqr.zero:
        raise ValueError(f"The additive identity element, {elem_name}, does not have an inverse.")
    else:
        delimiter = algebra.direct_product_delimiter()
        absvalsqr = cda_sqr_abs_val(elem_name, algebra, alg_sqr)
        absvalsqrinv = algebra.mult_inv(absvalsqr)
        return cda_scalar_mult(absvalsqrinv, cda_conjugate(elem_name, algebra), algebra)

## Code Tests

In [4]:
tests = ['2', '1:2', '2:1:0:2', '0:0']

In [5]:
Fn = F3
FnSqr = F3sqr
FnQuad = F3quad

In [6]:
scalar = '2'
for test in tests:
    print(f"{scalar} x {test} = {cda_scalar_mult(scalar, test, Fn)}")

2 x 2 = 1
2 x 1:2 = 2:1
2 x 2:1:0:2 = 1:2:0:1
2 x 0:0 = 0:0


In [7]:
for test in tests:
    print(f"-({test}) = {cda_negate(test, Fn)}")

-(2) = 1
-(1:2) = 2:1
-(2:1:0:2) = 1:2:0:1
-(0:0) = 0:0


In [8]:
for test in tests:
    print(f"conj({test}) = {cda_conjugate(test, Fn)}")

conj(2) = 2
conj(1:2) = 1:1
conj(2:1:0:2) = 2:2:0:1
conj(0:0) = 0:0


In [9]:
for test in tests[:2]: # just look at the first two items in tests
    print(f"|{test}|^2 = {cda_sqr_abs_val(test, Fn, FnSqr)}")

|2|^2 = 1
|1:2|^2 = 2


In [10]:
for test in tests[1:2]: # just look at the first two items in tests
    print(f"inv({test}) = {cda_inverse(test, Fn, FnSqr)}")

inv(1:2) = 2:2


In [11]:
FnSqr.mult_inv(tests[1])  # The better way to obtain an inverse

'2:2'

In [12]:
FnSqr.mult(tests[1], cda_inverse(tests[1], Fn, FnSqr))

'1:0'

In [13]:
try:
    cda_inverse('0:0', Fn, FnSqr)
except Exception as exc:
    print(exc)

The additive identity element, 0:0, does not have an inverse.


## String Manipulation

In [14]:
import re

In [15]:
def split_element(element, delimiter=':'):
    if delimiter in element:
        matches = list(re.finditer(delimiter, element))
        mid = matches[len(matches) // 2]
        return element[:mid.start()], element[mid.end():]
    else:
        return element

**split_element** tests:

In [16]:
x, y = split_element('a:b:c:d')
x, y

('a:b', 'c:d')

In [17]:
u, v = split_element(x)
u, v

('a', 'b')

In [18]:
split_element('a:b:c:d:e:f:g:h')

('a:b:c:d', 'e:f:g:h')

In [19]:
split_element('a')

'a'

In [20]:
split_element('')

''

## Cayley-Dickson Multiplication

According to [Shafer, 1966]

$a,b,c,d,\mu \in F$, $\mu \neq 0$, then $(a, b) \times (c, d) = (a c + \mu d \overline{b}, \overline{a} d + c b)$,

where $\overline{a} = a$ and $\overline{(a, b)} = (\overline{a}, -b)$

In [21]:
def conj(x, rng):
    delim = rng.direct_product_delimiter()
    if delim in x:
        a, b = split_element(x, delim)
        return conj(a, rng), rng.inv(b)
    else:
        return x

In [30]:
conj('1', F7)
conj('1:2', F7)
conj('1:2:3:4', F7sqr)

('1', '5')

In [22]:
def multiply(x, y, rng, mu=None):
    if mu is None:
        mu = rng.inv(rng.one)  # The additive inverse of the Ring's multiplicative identity
    delim = rng.direct_product_delimiter()
    if delim in x:
        a, b = split_element(x, delim)
        c, d = split_element(y, delim)
        return delim.join((rng.add(rng.mult(a, c), rng.mult(mu, d, conj(b, rng))),
                           rng.add(rng.mult(conj(a, rng), d), rng.mult(c, b))))
    else:
        return rng.mult(x, y)

In [23]:
x1 = '5:3'
y1 = '2:4'

In [24]:
x1y1 = multiply(x1, y1, F7)
print(f"{x1} * {y1} = {x1y1}")
print(F7sqr.mult(x1, y1))

x1y1 == F7sqr.mult(x1, y1)

5:3 * 2:4 = 5:5
5:5


True

In [25]:
All_OK = True
count = 0

for x1 in F7sqr.elements:
    for y1 in F7sqr.elements:
        count += 1
        xy_sqr = F7sqr.mult(x1, y1)
        xy_str = multiply(x1, y1, F7)
        if not xy_sqr == xy_str:
            print(x1, y1, xy_sqr, xy_str)
            All_OK = False

print(f"All {count} products OK? {All_OK}")

All 2401 products OK? True


In [26]:
x2 = '5'
y2 = '2'

In [27]:
x2y2 = multiply(x2, y2, F7)
print(f"{x2} * {y2} = {x2y2}")
#print(F7sqr.mult(x1, y1))

#x1y1 == F7sqr.mult(x1, y1)

5 * 2 = 3
